# Image-Based Model Drift Detection with CIFAR-10

## Description of Method

This notebook will run through an example of the model drift detection pipeline described [here](https://github.com/ctc-oss/ai_assurance_sandbox/tree/model-drift-detection/model_drift_detection).

To concretely detect model drift on the open-source CIFAR-10 dataset, we will create a base model and a production model from two different portions of the dataset. Then, using both model inversion and membership inference attacks, we will determine whether the base model's general idea of a feature or class has drifted, even if the test accuracy scores do not indicate that.

<sub>**Note**: This notebook was validated 5 September, 2023 in Google Colab.</sub>

# Detecting Model Drift Example

Let's Begin!

In [ ]:
!pip install adversarial-robustness-toolbox # version 1.15.1
!pip install pyyaml h5py  # Required to save models in HDF5 format

import os
import sys
import sklearn
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# dataset management
from tensorflow.keras.datasets import cifar10
tfds.image_classification.EMNIST
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# to initialize model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, \
    BatchNormalization, AveragePooling2D, Add, Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

# for model training
from tensorflow.keras.losses import categorical_crossentropy

# for model evaluation
from sklearn.metrics import classification_report

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 53.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


### Data Management

As a first data management step, we must establish the proper configuration for the dataset and the model.

*For a description of these variables, see [Configuration Notes](#Configuration-Notes).*

In [ ]:
# DATASET CONFIGURATION
num_classes = 10
image_size = 32
image_channels = 3
conv_filters = [
    32,
    64
]
dense_units = 1028
mode = "natural"
depth = 1
# TRAINING CONFIGURATION
EPOCHS = 25
batch_size = 128
train_size = 0.2
test_size = 0.2
examples_per_class = 100

Now, we load in the dataset and assign the training and testing data.

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


170498071/170498071 [==============================] - 14s 0us/step


Here, we split the data into the base training data (**x_base, y_base**) and the production training data (**x_production, y_production**).

In [ ]:
# reshape y_train
def flatten_y(y_temp):
    y_flatten = []
    for y_idx in range(len(y_temp)):
        y_flatten.append(np.argmax(y_temp[y_idx]))
    y_flatten = np.array(y_flatten)
    return y_flatten

x_base = x_train
y_base = y_train

y_flatten = flatten_y(y_base)
ids = np.array(list(range(len(y_flatten))))

ids_base, ids_prod = train_test_split(ids, train_size=train_size, test_size=test_size, random_state=None, shuffle = True, stratify=y_flatten)

x_base = x_train[ids_base]
y_base = y_train[ids_base]
x_production = x_train[ids_prod]
y_production = y_train[ids_prod]

Let's make sure that our training and testing shapes for the base and production data are what we expect...

In [ ]:
print("original training data x and y shapes:")
print(x_train.shape)
print(y_train.shape)

print("base training data x and y shapes:")
print(x_base.shape)
print(y_base.shape)

print("drifted production training data x and y shapes:")
print(x_production.shape)
print(y_production.shape)

print("testing data x and y shapes:")
print(x_test.shape)
print(y_test.shape)

original training data x and y shapes:
(50000, 32, 32, 3)
(50000, 10)
base training data x and y shapes:
(10000, 32, 32, 3)
(10000, 10)
drifted production training data x and y shapes:
(10000, 32, 32, 3)
(10000, 10)
testing data x and y shapes:
(10000, 32, 32, 3)
(10000, 10)


## Model Creation

Now, we have access to our train and test data in the correctly loaded configuration. The next step is to create the base model using **x_base** and **y_base**.

Here, we set the hyperparameters based on the model type and configuration settings and intialize the model with these hyperparameters.

In [ ]:
# set hyperparameters for model initialization
input_shape = (image_size, image_size, image_channels)
kernel_size = [3]
strides = [(1, 1)]
padding = ['valid', 'valid']
pool_size = (2, 2)
pool_strides = None
if isinstance(dense_units, int):
    dense_units = [dense_units]
num_fc_units = dense_units
dropout = [0.25, 0.25, 0.5]
embedding = None
adv_multiplier = 0.2
adv_step_size = 0.2
adv_grad_norm = 'infinity'
image_input_name = 'image'
transfer_learning = False
trainable = True

# initialize model
inputs = Input(shape=input_shape, dtype=tf.float32, name=image_input_name)
x = inputs
x = Conv2D(conv_filters[0], kernel_size=kernel_size[0], kernel_initializer='he_normal')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = Conv2D(conv_filters[1], kernel_size=kernel_size[0], kernel_initializer='he_normal')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=pool_size)(x)
x = Flatten()(x)
for num_units in num_fc_units:
    x = Dense(num_units, activation='relu')(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = Dense(num_classes, activation='linear', name='logits')(x)
pred = Activation('softmax', name="Softmax")(x)

### Create Base Model

In [ ]:
model_base = tf.keras.Model(inputs=inputs, outputs=pred, name="keras_generator_base")

loss = categorical_crossentropy
optimizer = Adam(learning_rate=1e-4, beta_1=1e-6)

model_base.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

model_base.summary()

Model: "keras_generator_base"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 activation (Activation)     (None, 30, 30, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 30, 30, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 activation_1 (Activation)   (None, 28, 28, 64)        0         
                                              

Now, we use Keras' *model.fit* function for the given epochs to create the base model.

In [ ]:
model_base.fit(x_base, y_base, batch_size=batch_size, epochs=EPOCHS, validation_data=(x_test, y_test))
model_base.save('model_base.h5', save_format='h5')

Epoch 1/25
79/79 [==============================] - 21s 38ms/step - loss: 1.8376 - accuracy: 0.4167 - val_loss: 2.3753 - val_accuracy: 0.2490
Epoch 2/25
79/79 [==============================] - 1s 19ms/step - loss: 0.8150 - accuracy: 0.7313 - val_loss: 2.2430 - val_accuracy: 0.2344
Epoch 3/25
79/79 [==============================] - 2s 23ms/step - loss: 0.4144 - accuracy: 0.8911 - val_loss: 2.3978 - val_accuracy: 0.2735
Epoch 4/25
79/79 [==============================] - 2s 21ms/step - loss: 0.2156 - accuracy: 0.9604 - val_loss: 1.8604 - val_accuracy: 0.3833
Epoch 5/25
79/79 [==============================] - 2s 23ms/step - loss: 0.0923 - accuracy: 0.9939 - val_loss: 1.6435 - val_accuracy: 0.4678
Epoch 6/25
79/79 [==============================] - 1s 19ms/step - loss: 0.0538 - accuracy: 0.9987 - val_loss: 1.4979 - val_accuracy: 0.5129
Epoch 7/25
79/79 [==============================] - 2s 22ms/step - loss: 0.0351 - accuracy: 0.9998 - val_loss: 1.3760 - val_accuracy: 0.5570
Epoch 8/25
7

In [ ]:
print('train loss and accuracy:')
train_loss_base, train_acc_base = model_base.evaluate(x_base, y_base, verbose=1, steps=batch_size+1)

print('test loss and accuracy:')
test_loss_base, test_acc_base = model_base.evaluate(x_test, y_test, verbose=1, steps=batch_size+1)

print('per class reports:')
report_base = classification_report(np.argmax(y_test, axis=1), np.argmax(model_base.predict((x_test)), axis=1),
                               output_dict=True)
for key,value in report_base.items():
	print(key, ':', value)

train loss and accuracy:
129/129 [==============================] - 1s 5ms/step - loss: 0.0063 - accuracy: 0.9995
test loss and accuracy:
129/129 [==============================] - 1s 5ms/step - loss: 1.6507 - accuracy: 0.5657
per class reports:
313/313 [==============================] - 1s 3ms/step
0 : {'precision': 0.5637254901960784, 'recall': 0.69, 'f1-score': 0.6205035971223021, 'support': 1000}
1 : {'precision': 0.6990394877267876, 'recall': 0.655, 'f1-score': 0.6763035622096025, 'support': 1000}
2 : {'precision': 0.43686354378818737, 'recall': 0.429, 'f1-score': 0.43289606458123103, 'support': 1000}
3 : {'precision': 0.41646778042959426, 'recall': 0.349, 'f1-score': 0.3797606093579978, 'support': 1000}
4 : {'precision': 0.5234042553191489, 'recall': 0.492, 'f1-score': 0.5072164948453609, 'support': 1000}
5 : {'precision': 0.4827586206896552, 'recall': 0.448, 'f1-score': 0.46473029045643155, 'support': 1000}
6 : {'precision': 0.6021034180543383, 'recall': 0.687, 'f1-score': 0.641

### Create Production Model

Next, we train our production model with the partitioned production data.

In [ ]:
model_production = tf.keras.models.load_model('model_base.h5')

model_production.fit(x_production, y_production, batch_size=batch_size, epochs=EPOCHS, validation_data=(x_test, y_test))
model_production.save('model_production.h5', save_format='h5')

Epoch 1/25
79/79 [==============================] - 5s 29ms/step - loss: 1.6491 - accuracy: 0.5185 - val_loss: 2.3760 - val_accuracy: 0.4781
Epoch 2/25
79/79 [==============================] - 2s 23ms/step - loss: 0.4481 - accuracy: 0.8650 - val_loss: 1.4571 - val_accuracy: 0.5479
Epoch 3/25
79/79 [==============================] - 2s 19ms/step - loss: 0.1381 - accuracy: 0.9817 - val_loss: 1.4093 - val_accuracy: 0.5709
Epoch 4/25
79/79 [==============================] - 2s 22ms/step - loss: 0.0578 - accuracy: 0.9983 - val_loss: 1.2284 - val_accuracy: 0.6126
Epoch 5/25
79/79 [==============================] - 2s 29ms/step - loss: 0.0293 - accuracy: 0.9998 - val_loss: 1.3000 - val_accuracy: 0.5982
Epoch 6/25
79/79 [==============================] - 3s 35ms/step - loss: 0.0223 - accuracy: 0.9999 - val_loss: 1.3032 - val_accuracy: 0.6097
Epoch 7/25
79/79 [==============================] - 2s 19ms/step - loss: 0.0157 - accuracy: 1.0000 - val_loss: 1.2800 - val_accuracy: 0.6128
Epoch 8/25
79

In [ ]:
print('production train loss and accuracy:')
train_loss_production, train_acc_production = model_production.evaluate(x_production, y_production, verbose=1, steps=batch_size+1)

print('production test loss and accuracy:')
test_loss_production, test_acc_production = model_production.evaluate(x_test, y_test, verbose=1, steps=batch_size+1)

print('production per class reports:')
report_production = classification_report(np.argmax(y_test, axis=1), np.argmax(model_production.predict((x_test)), axis=1),
                               output_dict=True)
for key,value in report_production.items():
	print(key, ':', value)

production train loss and accuracy:
129/129 [==============================] - 1s 5ms/step - loss: 0.0019 - accuracy: 1.0000
production test loss and accuracy:
129/129 [==============================] - 1s 6ms/step - loss: 1.4560 - accuracy: 0.6130
production per class reports:
313/313 [==============================] - 1s 3ms/step
0 : {'precision': 0.6277573529411765, 'recall': 0.683, 'f1-score': 0.6542145593869733, 'support': 1000}
1 : {'precision': 0.7370030581039755, 'recall': 0.723, 'f1-score': 0.7299343765774862, 'support': 1000}
2 : {'precision': 0.5128805620608899, 'recall': 0.438, 'f1-score': 0.47249190938511326, 'support': 1000}
3 : {'precision': 0.45089285714285715, 'recall': 0.404, 'f1-score': 0.42616033755274263, 'support': 1000}
4 : {'precision': 0.5665280665280665, 'recall': 0.545, 'f1-score': 0.5555555555555556, 'support': 1000}
5 : {'precision': 0.5090047393364929, 'recall': 0.537, 'f1-score': 0.5226277372262774, 'support': 1000}
6 : {'precision': 0.6562228024369017, '

## Gray Data Generation
The next step in our model drift detection pipeline is to run a model inversion attack on the base and production models and create gray data from both.

First, let's import everything we need to run the model inversion and generate the gray data.

In [ ]:
# GrayDataGenerator imports
from __future__ import absolute_import, division, print_function, unicode_literals
from typing import Optional, TYPE_CHECKING
from tqdm import trange
from tensorflow import keras
from art.estimators.classification.classifier import ClassifierMixin, ClassGradientsMixin
from art.estimators.estimator import BaseEstimator
from art.attacks.attack import InferenceAttack
from art.utils import get_labels_np_array, check_and_transform_label_format
if TYPE_CHECKING:
    from art.utils import CLASSIFIER_CLASS_LOSS_GRADIENTS_TYPE

# model inversion
from art.estimators.classification import TensorFlowV2Classifier
from tensorflow.keras.losses import CategoricalCrossentropy

### **GrayDataGenerator** Class
To create the gray data, we first need to port the [MIFace implementation from the Adversarial Robustness Toolbox](https://github.com/Trusted-AI/adversarial-robustness-toolbox/blob/main/art/attacks/inference/model_inversion/mi_face.py) into our example. However, we cannot just import this class because there are a few things that we have changed to work in our test case, so we rename the class **GrayDataGenerator**.

Running the following code section will port that class to our example:

In [ ]:
class GrayDataGenerator(InferenceAttack):

    attack_params = InferenceAttack.attack_params + [
        "max_iter",
        "window_length",
        "threshold",
        "learning_rate",
        "batch_size",
        "verbose",
    ]

    _estimator_requirements = (BaseEstimator, ClassifierMixin, ClassGradientsMixin)

    def __init__(
            self,
            classifier: "CLASSIFIER_CLASS_LOSS_GRADIENTS_TYPE",
            max_iter: int = 10000,
            window_length: int = 100,
            threshold: float = 0.99,
            learning_rate: float = 0.1,
            batch_size: int = 1,
            verbose: bool = True,
            g_size: int = 100,
    ):

        super().__init__(estimator=classifier)

        self.max_iter = max_iter
        self.window_length = window_length
        self.threshold = threshold
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.verbose = verbose
        self.g_size = g_size

    def infer(self, x: Optional[np.ndarray], y: Optional[np.ndarray] = None, k_model=None, **kwargs) -> np.ndarray:
        if x is None and y is None:
            raise ValueError("Either `x` or `y` should be provided.")

        y = check_and_transform_label_format(y, self.estimator.nb_classes)
        if x is None:
            x = np.zeros((len(y),) + self.estimator.input_shape)
        if y is None:
            y = get_labels_np_array(self.estimator.predict(x, batch_size=self.batch_size))

        x_infer = x.astype(np.float32)
        gray_x = []
        gray_y = []

        # Compute inversions with implicit batching
        for batch_id in trange(
                int(np.ceil(x.shape[0] / float(self.batch_size))), desc="Model inversion", disable=not self.verbose
        ):
            batch_index_1, batch_index_2 = batch_id * self.batch_size, (batch_id + 1) * self.batch_size
            batch = x_infer[batch_index_1:batch_index_2]
            batch_labels = y[batch_index_1:batch_index_2]

            active = np.array([True] * len(batch))
            window = np.inf * np.ones((len(batch), self.window_length))

            i = 0
            acc = False
            g_num = 0

            while i < self.max_iter and sum(active) > 0 and g_num < self.g_size:
                grads = self.estimator.class_gradient(batch[active], np.argmax(batch_labels[active], axis=1))
                grads = np.reshape(grads, (grads.shape[0],) + grads.shape[2:])
                batch[active] = batch[active] + self.learning_rate * grads

                if self.estimator.clip_values is not None:
                    clip_min, clip_max = self.estimator.clip_values
                    batch[active] = np.clip(batch[active], clip_min, clip_max)

                cost = 1 - self.estimator.predict(batch)[np.arange(len(batch)), np.argmax(batch_labels, axis=1)]
                active = (cost <= self.threshold) + (cost >= np.max(window, axis=1))

                i_window = i % self.window_length
                window[::, i_window] = cost
                i = i + 1

                if acc:
                    if image_channels == 1:
                        x = np.reshape(batch, (image_size, image_size, 1))
                    gray_x.append(x)
                    gray_y.append(batch_index_1)
                    g_num = g_num + 1
                else:
                    new_gray = keras.utils.to_categorical(np.array([batch_index_1]), num_classes)
                    scores = k_model.evaluate(batch, new_gray, verbose=1)
                    if int(scores[1] == 1):
                        acc = True
                        x = np.reshape(batch, (image_size, image_size, image_channels))
                        gray_x.append(x)
                        gray_y.append(batch_index_1)
                        g_num = g_num + 1
            x_infer[batch_index_1:batch_index_2] = batch

        return np.array(gray_x), np.array(gray_y)

### Model Inversion Attack

Now we use the **GrayDataGenerator** class to run the model inversion against the base model to create the "base" gray data.

In [ ]:
restore_model_base = tf.keras.models.load_model('model_base.h5')
loss_object = CategoricalCrossentropy(from_logits=True)

krc_base = TensorFlowV2Classifier(
    model=restore_model_base,
    nb_classes=num_classes,
    loss_object=loss_object,
    input_shape=(image_size, image_size, image_channels),
    clip_values=(0, 1),
)

attack_base = GrayDataGenerator(krc_base, max_iter=1000, threshold=1., batch_size=1, g_size=100)

x_init_average = np.zeros((num_classes, image_size, image_size, image_channels)) + np.mean(x_test, axis=0)
y_gray_base = np.arange(num_classes)
class_gradient = krc_base.class_gradient(x_init_average, y_gray_base)
class_gradient = np.reshape(class_gradient, (num_classes, image_size * image_size, image_channels))
class_gradient_max = np.max(class_gradient, axis=1)

print("Minimum over all maximum class gradient: %f" % (np.min(class_gradient_max)))

Minimum over all maximum class gradient: 0.000000


To ensure that the gray data is recognizable by the base model, we reevaluate the base model with the new gray data. (This step should take ~5 min if you are running on a GPU.)

In [ ]:
x_gray_base, y_gray_base = attack_base.infer(x_init_average, y_gray_base, restore_model_base)
y_gray_base = keras.utils.to_categorical(np.array(y_gray_base), num_classes)

print('\n gray data loss and accuracy based on base model:')
scores_gray_base = restore_model_base.evaluate(x_gray_base, y_gray_base, verbose=1)

Model inversion:   0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 28ms/step - loss: 1.1921e-07 - accuracy: 1.0000


Model inversion:  10%|█         | 1/10 [00:03<00:29,  3.30s/it]

1/1 [==============================] - 0s 27ms/step - loss: 19.2184 - accuracy: 0.0000e+00


Model inversion:  20%|██        | 2/10 [01:51<08:41, 65.17s/it]

...

Model inversion:  90%|█████████ | 9/10 [05:52<00:46, 46.19s/it]

1/1 [==============================] - 0s 25ms/step - loss: 20.1980 - accuracy: 0.0000e+00


Model inversion: 100%|██████████| 10/10 [07:40<00:00, 46.07s/it]


 gray data loss and accuracy based on base model:
15/19 [======================>.......] - ETA: 0s - loss: 1.2388e-05 - accuracy: 1.0000

19/19 [==============================] - 0s 9ms/step - loss: 1.3252e-05 - accuracy: 1.0000


Let's do the same for the production model.

In [ ]:
restore_model_production = tf.keras.models.load_model('model_production.h5')
loss_object = CategoricalCrossentropy(from_logits=True)

krc_production = TensorFlowV2Classifier(
    model=restore_model_production,
    nb_classes=num_classes,
    loss_object=loss_object,
    input_shape=(image_size, image_size, image_channels),
    clip_values=(0, 1),
)

attack_production = GrayDataGenerator(krc_production, max_iter=1000, threshold=1., batch_size=1, g_size=100)

x_init_average = np.zeros((num_classes, image_size, image_size, image_channels)) + np.mean(x_test, axis=0)
y_gray_production = np.arange(num_classes)
class_gradient = krc_production.class_gradient(x_init_average, y_gray_production)
class_gradient = np.reshape(class_gradient, (num_classes, image_size * image_size, image_channels))
class_gradient_max = np.max(class_gradient, axis=1)

print("Minimum over all maximum class gradient: %f" % (np.min(class_gradient_max)))

Minimum over all maximum class gradient: 0.000000


To ensure that the gray data is recognizable by the production model, we reevaluate the production model with the new gray data. (This step should take ~5 min if you are running on a GPU.)

In [ ]:
x_gray_production, y_gray_production = attack_production.infer(x_init_average, y_gray_production, restore_model_production)
y_gray_production = keras.utils.to_categorical(np.array(y_gray_production), num_classes)

print('\n gray data loss and accuracy based on production model:')
scores_gray_production = restore_model_production.evaluate(x_gray_production, y_gray_production, verbose=1)
print('Gray data loss:', scores_gray_production[0])
print('Gray data accuracy:', scores_gray_production[1])

Model inversion:   0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 42ms/step - loss: 0.0025 - accuracy: 1.0000


Model inversion:  10%|█         | 1/10 [00:38<05:48, 38.72s/it]

1/1 [==============================] - 0s 37ms/step - loss: 33.8819 - accuracy: 0.0000e+00


Model inversion:  20%|██        | 2/10 [02:25<10:28, 78.61s/it]

...

Model inversion:  90%|█████████ | 9/10 [12:17<01:07, 67.55s/it]

1/1 [==============================] - 0s 27ms/step - loss: 31.7334 - accuracy: 0.0000e+00


Model inversion: 100%|██████████| 10/10 [14:07<00:00, 84.78s/it]


 gray data loss and accuracy based on production model:
 1/10 [==>...........................] - ETA: 0s - loss: 2.3589e-04 - accuracy: 1.0000

10/10 [==============================] - 0s 13ms/step - loss: 1.4590e-04 - accuracy: 1.0000
Gray data loss: 0.0001458962942706421
Gray data accuracy: 1.0


## Membership Inference Attacks

Now that we have our gray data from both the base model and production model, we will use this data to check whether the base model has experienced model drift from the base to the production data.


We take both the rule-based membership inference (MI) attack and the black-box MI attack from the [Adversarial Robustness Toolbox example](https://github.com/Trusted-AI/adversarial-robustness-toolbox/blob/main/notebooks/attack_membership_inference_shadow_models.ipynb).

In [ ]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox, MembershipInferenceBlackBoxRuleBased
from art.utils import to_categorical
from art.estimators.classification import TensorFlowV2Classifier

restore_model_base2 = tf.keras.models.load_model('model_base.h5')

art_classifier = TensorFlowV2Classifier(restore_model_base2, nb_classes=num_classes, input_shape=(image_size, image_size, image_channels))

###Rule-Based Membership Inference Attack

In [ ]:
rb_attack = MembershipInferenceBlackBoxRuleBased(art_classifier)
inferred_test = rb_attack.infer(x_test, y_test)
rb_test_acc = 1 - (np.sum(inferred_test) / len(inferred_test))
print(f"Test Data Accuracy {rb_test_acc:.4f}")

Test Data Accuracy 0.4343


Here we infer membership of the gray data created from the base model, which should provide a high member accuracy.

In [ ]:
inferred_train = rb_attack.infer(x_gray_base, y_gray_base)

rb_base_train_acc = np.sum(inferred_train) / len(inferred_train)
rb_base_acc = (rb_base_train_acc * len(inferred_train) + rb_test_acc * len(inferred_test)) / (len(inferred_train) + len(inferred_test))
print(f"Base Gray Data Accuracy: {rb_base_train_acc:.4f}")
print(f"Base Attack Accuracy {rb_base_acc:.4f}")

Base Gray Data Accuracy: 1.0000
Base Attack Accuracy 0.4663


Now we infer membership of the gray data created from the production model; this accuracy should be lower that the previous if model drift occured.

In [ ]:
prod_inferred_train = rb_attack.infer(x_gray_production, y_gray_production)

rb_prod_train_acc = np.sum(prod_inferred_train) / len(prod_inferred_train)
rb_prod_acc = (rb_prod_train_acc * len(prod_inferred_train) + rb_test_acc * len(inferred_test)) / (len(prod_inferred_train) + len(inferred_test))
print(f"Production Gray Data Accuracy: {rb_prod_train_acc:.4f}")
print(f"Production Attack Accuracy {rb_prod_acc:.4f}")

Production Gray Data Accuracy: 0.6667
Production Attack Accuracy 0.4411


In [ ]:
print("Rule-Base Model Drift Detection Result:")
result_a = (rb_prod_acc < rb_base_acc)
if result_a:
  rb_score = rb_base_acc - rb_prod_acc
  print(f"drift: {rb_score:.4f}")
else:
  print("no drift detected")

print("Rule-Base Concept Drift Detection Result:")
result_b = (rb_prod_train_acc < rb_base_train_acc)
if result_b:
  rb_score_c = rb_base_train_acc - rb_prod_train_acc
  print(f"drift: {rb_score_c:.4f}")
else:
  print("no drift detected")

Rule-Base Model Drift Detection Result:
True
ruled-based drift score: 0.0253
Rule-Base Concept Drift Detection Result:
True
ruled-based drift score: 0.3333


###Black-Box Membership Inference Attack

We run the black-box membership inference attack in the same way as the rule-based attack after fitting the attack model to the base training data that was used to fit the base model originally.

In [ ]:
bb_attack_bb = MembershipInferenceBlackBox(art_classifier)
bb_attack_bb.fit(x_base, y_base, x_test, y_test)

nonmember_infer = bb_attack_bb.infer(x_test, y_test)
bb_nonmember_acc = 1 - np.sum(nonmember_infer) / len(x_test)
print('Test Data Accuracy:', bb_nonmember_acc)

Test Data Accuracy: 0.8075


In [ ]:
base_member_infer = bb_attack_bb.infer(x_gray_base, y_gray_base)
bb_base_member_acc = np.sum(base_member_infer) / len(x_gray_base)
bb_base_acc = (bb_base_member_acc * len(x_gray_base) + bb_nonmember_acc * len(x_test)) / (len(x_gray_base) + len(x_test))
print('Base Gray Data Accuracy:', bb_base_member_acc)
print('Base Attack Accuracy:', bb_base_acc)

Base Gray Data Accuracy: 1.0
Base Attack Accuracy: 0.8183962264150944


In [ ]:
production_member_infer = bb_attack_bb.infer(x_gray_production, y_gray_production)
bb_production_member_acc = np.sum(production_member_infer) / len(x_gray_production)
bb_production_acc = (bb_production_member_acc * len(x_gray_production) + bb_nonmember_acc * len(x_test)) / (len(x_gray_production) + len(x_test))
print('Production Gray Data Accuracy:', bb_production_member_acc)
print('Production Attack Accuracy:', bb_production_acc)

Production Gray Data Accuracy: 0.3333333333333333
Production Attack Accuracy: 0.7936893203883495


In [ ]:
print("Black Box Model Drift Detection Result:")
result_c = (bb_production_acc < bb_base_acc)
if result_c:
  bb_score = bb_base_acc - bb_production_acc
  print(f"drift: {bb_score:.4f}")
else:
  print("no drift detected")

print("Black Box Concept Drift Detection Result:")
result_d = (bb_production_member_acc < bb_base_member_acc)
if result_d:
  bb_score_c = bb_base_member_acc - bb_production_member_acc
  print(f"drift: {bb_score_c:.4f}")
else:
  print("no drift detected")

Black Box Model Drift Detection Result:
True
black box drift score: 0.0247
Black Box Concept Drift Detection Result:
True
black box drift score: 0.6667


## Model Drift Detection

Let's take a look at the accuracy scores of the membership inference attack with the base gray data compared to the attack with the production gray data.

### Concept Drift Detection
If **production_member_acc** << **base_member_acc** for either attack, then we know that concept drift has occured.

In [ ]:
print("Concept Drift Detection Result:")
print((result_b | result_d))

Concept Drift Detection Result:
True


### Model Drift Detection
If **bb_production_acc** << **bb_base_acc** for either attack, then we believe that data drift has occured.

In [ ]:
print("Model Drift Detection Result:")
print((result_a | result_c))

Model Drift Detection Result:
True


# Configuration Notes

## Dataset Configuration

* **data_set**: name of data set mapped to utils.get_data

* **num_classes**: number of classes in dataset

* **image_size**: size of images in dataset

* **image_channels**: number of image channels in dataset

* **data_path**: path to saved data *(default "data")*

* **data_augmentation**: set to *True* if data augmentation should be used to train the model

## Model Configuration

* **conv_filters**: array for conv filters  *(ex. [4, 8])*

* **dense_units**: size of dense units

* **mode**: training mode *("natural", "robust", or "mini-robust")*

* **depth**:

## Training Configuration

* **epochs**: number of epochs

* **batch_size**:

* **train_size**: